In [41]:
from pymongo import MongoClient
import pandas as pd 
import re
from collections import Counter
from konlpy.tag import Okt
from mecab import MeCab
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer # word embedding

### TF-IDF 사용 문장 유사도 작성

In [42]:
it_sentences = [
    "인공지능 기술이 사회의 다양한 분야에 점점 더 널리 적용되고 있다.",
    "클라우드 컴퓨팅은 데이터 저장과 처리를 더욱 효율적으로 만들어 주며, 기업의 IT 인프라를 혁신하고 있다.",
    "머신러닝과 딥러닝 알고리즘은 데이터 분석과 예측 모델링에 혁명을 일으키고 있다.",
    "블록체인 기술은 금융 거래의 투명성과 보안성을 향상시키는데 기여하고 있다.",
    # "사이버 보안은 디지털 시대의 중요한 이슈로 부상하고 있으며, 지속적인 관심과 개선이 요구된다.",
    "소프트웨어 개발에서의 애자일 방법론은 빠른 시장 변화에 효과적으로 대응할 수 있게 해준다.",
    "데이터 과학과 분석은 비즈니스 의사 결정 과정에서 중요한 역할을 하며, 기업의 경쟁력을 강화시킨다.",
    "사물인터넷(IoT) 기술은 일상 생활을 더욱 편리하고 연결된 공간으로 변화시키고 있다.",
    "가상현실(VR)과 증강현실(AR) 기술은 엔터테인먼트, 교육, 의료 등 다양한 분야에서 새로운 경험을 제공한다.",
    "파이썬은 데이터 과학, 웹 개발, 자동화 스크립트 작성 등 다양한 프로그래밍 작업에 널리 사용되는 언어이다."
]

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer # word embedding

tfidfvectorizer = TfidfVectorizer()
tfidfvectorizer.fit(it_sentences) # 학습
result_vectors = tfidfvectorizer.transform(it_sentences) 

In [44]:
result_vectors.toarray().shape, tfidfvectorizer.get_feature_names_out().shape
# 문장 수 , 단어+gram 수

((9, 89), (89,))

In [45]:
result_vectors, tfidfvectorizer.get_feature_names_out()

(<9x89 sparse matrix of type '<class 'numpy.float64'>'
 	with 103 stored elements in Compressed Sparse Row format>,
 array(['ar', 'iot', 'it', 'vr', '가상현실', '강화시킨다', '개발', '개발에서의', '거래의',
        '결정', '경쟁력을', '경험을', '공간으로', '과정에서', '과학', '과학과', '교육', '금융',
        '기술은', '기술이', '기업의', '기여하고', '널리', '다양한', '대응할', '더욱', '데이터',
        '딥러닝', '만들어', '머신러닝과', '모델링에', '방법론은', '변화시키고', '변화에', '보안성을',
        '분석과', '분석은', '분야에', '분야에서', '블록체인', '비즈니스', '빠른', '사물인터넷', '사용되는',
        '사회의', '새로운', '생활을', '소프트웨어', '스크립트', '시장', '알고리즘은', '애자일', '언어이다',
        '엔터테인먼트', '역할을', '연결된', '예측', '의료', '의사', '인공지능', '인프라를', '일상',
        '일으키고', '있게', '있다', '자동화', '작성', '작업에', '저장과', '적용되고', '점점',
        '제공한다', '주며', '중요한', '증강현실', '처리를', '컴퓨팅은', '클라우드', '투명성과', '파이썬은',
        '편리하고', '프로그래밍', '하며', '해준다', '향상시키는데', '혁명을', '혁신하고', '효과적으로',
        '효율적으로'], dtype=object))

In [46]:
# 문장간 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
df_similarity = cosine_similarity(result_vectors, result_vectors)
df_similarity

array([[1.        , 0.0348656 , 0.04112589, 0.04336725, 0.        ,
        0.        , 0.03930179, 0.05633261, 0.13309856],
       [0.0348656 , 1.        , 0.07321871, 0.03422518, 0.        ,
        0.09329882, 0.0970222 , 0.        , 0.03530324],
       [0.04112589, 0.07321871, 1.        , 0.04037048, 0.        ,
        0.04084395, 0.03658595, 0.        , 0.04164211],
       [0.04336725, 0.03422518, 0.04037048, 1.        , 0.        ,
        0.        , 0.10064607, 0.05529789, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.09329882, 0.04084395, 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.03537386],
       [0.03930179, 0.0970222 , 0.03658595, 0.10064607, 0.        ,
        0.        , 1.        , 0.05011398, 0.        ],
       [0.05633261, 0.        , 0.        , 0.05529789, 0.        ,
        0.        , 0.05011398, 1.        , 0.04541681],


In [48]:
df_result = pd.DataFrame(data=df_similarity,index=it_sentences,columns=it_sentences)

In [51]:
new_sentence = ['사이버 보안은 디지털 시대의 중요한 이슈로 부상하고 있으며, 지속적인 관심과 개선이 요구된다.'
                , '클라우드 컴퓨팅은 데이터 저장과 처리를 더욱 효율적으로 만들어']

new_tfidfvectorizer = tfidfvectorizer.transform(new_sentence) # 입력 text를 vocablary를 기준으로 embedding 함
new_tfidfvectorizer

<2x89 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [53]:
similarity_array = cosine_similarity(new_tfidfvectorizer, result_vectors)
# 닮은것 찾기 어려움 그래서 많은 문장과 단어를 학습해 두어야 함.
similarity_array

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.28707214, 0.        , 0.        , 0.        ],
       [0.        , 0.76524671, 0.05326701, 0.        , 0.        ,
        0.04524891, 0.08625384, 0.        , 0.04613315]])

In [59]:
idx = similarity_array[0].argmax()
it_sentences[idx]

'데이터 과학과 분석은 비즈니스 의사 결정 과정에서 중요한 역할을 하며, 기업의 경쟁력을 강화시킨다.'

In [60]:
idx = similarity_array[1].argmax()
it_sentences[idx]

'클라우드 컴퓨팅은 데이터 저장과 처리를 더욱 효율적으로 만들어 주며, 기업의 IT 인프라를 혁신하고 있다.'